In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt

# --------- Parameter Settings ---------
content_img_path = "E:/CV/content.jpg"  # Replace with your own path
style_img_path = "E:/CV/style.jpg"      # Replace with your own path
output_img_path = "E:/CV/output.jpg"
device = torch.device("cpu")  # Use CPU only
image_size = 512
num_steps = 200  # Number of optimization steps
tv_weight = 1e-6  # Total variation loss weight

# --------- Image Preprocessing ---------
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[:3, :, :]),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def load_image(path):
    image = Image.open(path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image.to(device)

def im_convert(tensor):
    image = tensor.clone().detach()
    image = image.squeeze(0)
    image = image * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    image = image + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    image = image.clamp(0, 1)
    return transforms.ToPILImage()(image)

content = load_image(content_img_path)
style = load_image(style_img_path)

# --------- Feature Extraction Network ---------
class VGGFeatures(nn.Module):
    def __init__(self):
        super(VGGFeatures, self).__init__()
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(device).eval()
        self.layers = {'0': 'conv1_1', '5': 'conv2_1', 
               '10': 'conv3_1', '19': 'conv4_1', '28': 'conv5_1'}

    def forward(self, x):
        features = {}
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.layers:
                features[self.layers[name]] = x
        return features

# --------- Gram Matrix Computation ---------
def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(b * c, h * w)
    G = torch.mm(features, features.t())
    return G.div(b * c * h * w)

# --------- Initialization ---------
model = VGGFeatures().to(device)
target = torch.randn_like(content).to(device)
target.requires_grad = True
optimizer = optim.Adam([target], lr=0.05)
style_features = model(style)
content_features = model(content)

style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# Style and content weights
style_weights = {
    'conv1_1': 0.2,
    'conv2_1': 0.2,
    'conv3_1': 0.2,
    'conv4_1': 0.2
}
content_weight = 1  # 降低内容权重
style_weight = 1e5  # 提高风格权重
# --------- Optimization Loop ---------
for step in range(num_steps):
    optimizer.zero_grad()
    target_features = model(target)

    # Content loss
    content_loss = torch.mean((target_features['conv4_1'] - content_features['conv4_1'])**2)

    # Style loss
    style_loss = 0
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        style_loss += style_weights[layer] * torch.mean((target_gram - style_gram)**2)

    # Total variation loss for smoothness
    tv_loss = torch.sum(torch.abs(target[:, :, :, :-1] - target[:, :, :, 1:])) + \
              torch.sum(torch.abs(target[:, :, :-1, :] - target[:, :, 1:, :]))

    total_loss = content_weight * content_loss + style_weight * style_loss + tv_weight * tv_loss
    total_loss.backward(retain_graph=True) 
    optimizer.step()

    if step % 50 == 0:
        print(f"Step {step}, Total loss {total_loss.item():.2f}")

# --------- Save and Display Output ---------
output_img = im_convert(target)
output_img.save(output_img_path)
output_img.show()


Step 0, Total loss 149.59
Step 50, Total loss 11.42
Step 100, Total loss 7.69
Step 150, Total loss 6.17


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt

# --------- Parameter Settings ---------
content_img_path = "E:/CV/content.jpg"  # Replace with your own path
style_img_path = "E:/CV/style.jpg"      # Replace with your own path
output_img_path = "E:/CV/output.jpg"

# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Improved parameters for better quality
image_size = 512  # You can increase this for higher resolution (e.g., 1024)
num_steps = 300   # More steps for better convergence
tv_weight = 1e-6  # Total variation loss weight

# --------- Image Preprocessing ---------
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[:3, :, :]),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def load_image(path):
    image = Image.open(path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image.to(device)

def im_convert(tensor):
    image = tensor.clone().detach().cpu()  # Move to CPU for PIL conversion
    image = image.squeeze(0)
    image = image * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    image = image + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    image = image.clamp(0, 1)
    return transforms.ToPILImage()(image)

# Load images
content = load_image(content_img_path)
style = load_image(style_img_path)

# --------- Feature Extraction Network ---------
class VGGFeatures(nn.Module):
    def __init__(self):
        super(VGGFeatures, self).__init__()
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(device).eval()
        # Freeze VGG parameters
        for param in self.vgg.parameters():
            param.requires_grad_(False)
            
        self.layers = {'0': 'conv1_1', '5': 'conv2_1', 
               '10': 'conv3_1', '19': 'conv4_1', '28': 'conv5_1'}
    
    def forward(self, x):
        features = {}
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.layers:
                features[self.layers[name]] = x
        return features

# --------- Gram Matrix Computation ---------
def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(b * c, h * w)
    G = torch.mm(features, features.t())
    return G.div(b * c * h * w)

# --------- Initialization ---------
model = VGGFeatures().to(device)

# Initialize with content image for faster convergence
target = content.clone().requires_grad_(True).to(device)

# Use LBFGS optimizer for better results
optimizer = optim.LBFGS([target])

style_features = model(style)
content_features = model(content)
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# Style and content weights
style_weights = {
    'conv1_1': 0.75,  # More weight on lower layers for fine details
    'conv2_1': 0.5,
    'conv3_1': 0.2,
    'conv4_1': 0.2,
    'conv5_1': 0.2,   # Added deeper layer
}

content_weight = 1      
style_weight = 1e6      # Increased style weight for better stylization

# --------- Optimization Loop ---------
step = [0]  # Use a list to track step in closure
def closure():
    optimizer.zero_grad()
    target_features = model(target)
    
    # Content loss - only use one deeper layer
    content_loss = torch.mean((target_features['conv4_1'] - content_features['conv4_1'])**2)
    
    # Style loss
    style_loss = 0
    for layer in style_weights:
        if layer in target_features:  # Make sure layer exists
            target_feature = target_features[layer]
            target_gram = gram_matrix(target_feature)
            style_gram = style_grams[layer]
            layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
            style_loss += layer_style_loss
    
    # Total variation loss for smoothness
    tv_loss = torch.sum(torch.abs(target[:, :, :, :-1] - target[:, :, :, 1:])) + \
              torch.sum(torch.abs(target[:, :, :-1, :] - target[:, :, 1:, :]))
    
    total_loss = content_weight * content_loss + style_weight * style_loss + tv_weight * tv_loss
    total_loss.backward()
    
    step[0] += 1
    if step[0] % 20 == 0:
        print(f"Step {step[0]}, Total loss: {total_loss.item():.2f}")
    
    return total_loss

# Run optimization with LBFGS
for i in range(num_steps // 10):  # LBFGS takes bigger steps
    optimizer.step(closure)
    
    # Save intermediate results
    if (i+1) % 5 == 0:
        with torch.no_grad():
            output_img = im_convert(target)
            output_img.save(f"E:/CV/output_step_{(i+1)*10}.jpg")

# --------- Save and Display Output ---------
output_img = im_convert(target)
output_img.save(output_img_path)

# Option to upscale the final image for better clarity
try:
    from PIL import Image
    final_img = Image.open(output_img_path)
    upscaled_img = final_img.resize((final_img.width*2, final_img.height*2), Image.LANCZOS)
    upscaled_img.save("E:/CV/output_upscaled.jpg")
    print("Created upscaled version at E:/CV/output_upscaled.jpg")
except Exception as e:
    print(f"Could not create upscaled version: {e}")

# Display the final image
plt.figure(figsize=(10, 10))
plt.imshow(output_img)
plt.title("Final Style Transfer Result")
plt.axis('off')
plt.show()

Using device: cpu
Step 20, Total loss: 16.94
Step 40, Total loss: 13.13
Step 60, Total loss: 12.02
Step 80, Total loss: 11.50
Step 100, Total loss: 11.18
Step 120, Total loss: 10.97
Step 140, Total loss: 10.82
Step 160, Total loss: 10.71
Step 180, Total loss: 10.63
Step 200, Total loss: 10.56
Step 220, Total loss: 10.50
Step 240, Total loss: 10.45
Step 260, Total loss: 10.41
Step 280, Total loss: 10.38
Step 300, Total loss: 10.35
Step 320, Total loss: 10.32
Step 340, Total loss: 10.30
Step 360, Total loss: 10.28
Step 380, Total loss: 10.26
Step 400, Total loss: 10.24
Step 420, Total loss: 10.22
Step 440, Total loss: 10.21
Step 460, Total loss: 10.20
Step 480, Total loss: 10.18
Step 500, Total loss: 10.17
Step 520, Total loss: 10.16
Step 540, Total loss: 10.15
Step 560, Total loss: 10.14
Step 580, Total loss: 10.14
Step 600, Total loss: 10.13
Created upscaled version at E:/CV/output_upscaled.jpg
